In [1]:
import torch
import torchvision
import torchvision.models as models

vgg16 = models.vgg16(pretrained=True)

/Users/AlexHome/vms/effdl-inference/07-software-inference/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
example = torch.rand(1, 3, 224, 224)

In [3]:
torch.onnx.export(vgg16, example, "vgg16.onnx", verbose=True)

graph(%input.1 : Float(1, 3, 224, 224, strides=[150528, 50176, 224, 1], requires_grad=0, device=cpu),
      %features.0.weight : Float(64, 3, 3, 3, strides=[27, 9, 3, 1], requires_grad=1, device=cpu),
      %features.0.bias : Float(64, strides=[1], requires_grad=1, device=cpu),
      %features.2.weight : Float(64, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=1, device=cpu),
      %features.2.bias : Float(64, strides=[1], requires_grad=1, device=cpu),
      %features.5.weight : Float(128, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=1, device=cpu),
      %features.5.bias : Float(128, strides=[1], requires_grad=1, device=cpu),
      %features.7.weight : Float(128, 128, 3, 3, strides=[1152, 9, 3, 1], requires_grad=1, device=cpu),
      %features.7.bias : Float(128, strides=[1], requires_grad=1, device=cpu),
      %features.10.weight : Float(256, 128, 3, 3, strides=[1152, 9, 3, 1], requires_grad=1, device=cpu),
      %features.10.bias : Float(256, strides=[1], requires_grad=1, device

In [ ]:
! head vgg16.onnx

In [ ]:
! pip install onnx tensorflow==2.11.0 tensorflow-addons tensorflow-probability onnx_tf

In [4]:
import onnx
from onnx_tf.backend import prepare

2023-03-06 16:27:18.811891: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
model = onnx.load('vgg16.onnx')
tf_rep = prepare(model) 

2023-03-06 16:27:49.832462: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
! wget https://upload.wikimedia.org/wikipedia/commons/thumb/3/3a/Cat03.jpg/1200px-Cat03.jpg

--2023-03-06 16:28:09--  https://upload.wikimedia.org/wikipedia/commons/thumb/3/3a/Cat03.jpg/1200px-Cat03.jpg
Распознаётся upload.wikimedia.org (upload.wikimedia.org)… 2620:0:862:ed1a::2:b, 91.198.174.208
Подключение к upload.wikimedia.org (upload.wikimedia.org)|2620:0:862:ed1a::2:b|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 156145 (152K) [image/jpeg]
Сохранение в: «1200px-Cat03.jpg.3»

1200px-Cat03.jpg.3  100%[===================>] 152,49K  --.-KB/s    за 0,1s    

2023-03-06 16:28:10 (1,02 MB/s) - «1200px-Cat03.jpg.3» сохранён [156145/156145]



In [7]:
import torchvision.transforms as transforms

transform_pipeline = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
])


def prepare(img_path):
    image = Image.open(img_path)
    img_data = transform_pipeline(image).unsqueeze(0)
    data = np.array(img_data)
    return data

In [9]:
import numpy as np
from IPython.display import display
from PIL import Image

img = prepare('1200px-Cat03.jpg')

In [10]:
output = tf_rep.run(np.asarray(img, dtype=np.float32))

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [11]:
np.argmax(output)

285

In [12]:
tf_rep.export_graph('vgg16.pb')

INFO:tensorflow:Assets written to: vgg16.pb/assets


INFO:tensorflow:Assets written to: vgg16.pb/assets


In [13]:
! ls

1200px-Cat03.jpg   1200px-Cat03.jpg.2 README.md          vgg16.onnx
1200px-Cat03.jpg.1 1200px-Cat03.jpg.3 onnx.ipynb         vgg16.pb


In [14]:
! ls vgg16.pb

assets         fingerprint.pb saved_model.pb variables


In [15]:
import tensorflow as tf

In [16]:
modeltf = tf.saved_model.load("vgg16.pb")

In [17]:
modeltf

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject at 0x182d74d00>

In [18]:
infer = modeltf.signatures["serving_default"]

In [19]:
tf.constant(img)

<tf.Tensor: shape=(1, 3, 224, 224), dtype=float32, numpy=
array([[[[1.5981677 , 1.6324171 , 1.6837914 , ..., 1.8379141 ,
          1.8550389 , 1.8550389 ],
         [1.5981677 , 1.6324171 , 1.6837914 , ..., 1.8721637 ,
          1.8892884 , 1.8892884 ],
         [1.5981677 , 1.6324171 , 1.6837914 , ..., 1.9064132 ,
          1.9064132 , 1.8892884 ],
         ...,
         [1.1015497 , 1.1357993 , 1.0844251 , ..., 1.6324171 ,
          1.6324171 , 1.6152923 ],
         [1.1186745 , 1.1529241 , 1.1015497 , ..., 1.6837914 ,
          1.6837914 , 1.6837914 ],
         [1.0501755 , 1.0844251 , 1.0844251 , ..., 1.7180408 ,
          1.7180408 , 1.7351656 ]],

        [[1.6232493 , 1.6757703 , 1.7457983 , ..., 1.8858544 ,
          1.8683473 , 1.8508403 ],
         [1.6057423 , 1.6757703 , 1.7457983 , ..., 1.8858544 ,
          1.8858544 , 1.8508403 ],
         [1.6407562 , 1.7107843 , 1.7633053 , ..., 1.8858544 ,
          1.8508403 , 1.7983193 ],
         ...,
         [0.71288526, 0.765406

In [20]:
output = infer(tf.constant(img))

In [21]:
output

{'70': <tf.Tensor: shape=(1, 1000), dtype=float32, numpy=
 array([[-5.11787605e+00, -9.00458515e-01, -1.33540070e+00,
         -3.37574929e-02, -1.62641752e+00,  1.24357831e+00,
         -1.14815104e+00,  1.55836046e-02,  2.63606381e+00,
         -3.83762860e+00, -1.54843211e+00, -2.95213914e+00,
          1.23569787e+00, -7.08229601e-01, -3.13737106e+00,
         -7.40747750e-01, -4.38672811e-01,  1.04632415e-01,
         -2.55209517e+00, -3.68814731e+00, -2.86074519e+00,
         -3.22845554e+00, -4.80303049e+00, -3.59836626e+00,
         -2.71968842e+00, -4.30782700e+00, -4.68665552e+00,
         -2.75465727e+00, -4.50009394e+00, -2.30202460e+00,
         -1.76955879e+00, -2.61921310e+00, -2.64150381e+00,
         -3.41213989e+00, -3.77243876e+00, -4.06072855e+00,
         -1.76341259e+00, -3.37075162e+00, -7.38505781e-01,
         -2.89529276e+00, -6.44332469e-01, -2.17674971e+00,
         -2.18159246e+00, -2.43516016e+00, -1.87657857e+00,
         -4.09389210e+00, -1.22118938e+00,

In [22]:
values = next(iter(output.values()))

In [23]:
np.argmax(values)

285